In [ ]:
%cd yolov5/
!python export.py --weights runs/train/yolov5s_results4/weights/best.pt --include onnx

In [ ]:
import cv2
import numpy as np

import os
from os import listdir
from os.path import isfile, join
curr_dir = os.getcwd()

if curr_dir.endswith("yolov5"):
    os.chdir('..')


dataset_location = "trash_detection.v2i.yolov5pytorch"
train_folder=dataset_location + "/train"
test_folder=dataset_location + "/valid"

# read number of classes based on YAML config file:
import yaml
with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    num_classes = str(data_cfg['nc'])
    list_classes = data_cfg['names']
    print("Dataset defines", num_classes, "classes ->\n", list_classes)
    
train_imgs = [f for f in listdir(train_folder+"/images") if (isfile(join(train_folder+"/images", f)) and f.endswith(".jpg"))]
test_imgs = [f for f in listdir(test_folder+"/images") if (isfile(join(test_folder+"/images", f)) and f.endswith(".jpg"))]


# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:


# read input image
image = join(train_folder+"/images", train_imgs[0])
image = cv2.imread(image)


Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(list_classes), 3))

# read pre-trained model and config file
net = cv2.dnn.readNet("yolov5/runs/train/yolov5s_results4/weights/best.onnx", "yolov5/models/custom_yolov5s.yaml")

is_cuda = False
if is_cuda:
    print("Attempty to use CUDA")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
else:
    print("Running on CPU")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# create input blob 
blob = cv2.dnn.blobFromImage(image, scale, (Width,Height), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

In [ ]:
INPUT_WIDTH = Width
INPUT_HEIGHT = Height
SCORE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.4
CONFIDENCE_THRESHOLD = 0.4



def format_yolov5(frame):

    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

def detect(image, net):
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (INPUT_WIDTH, INPUT_HEIGHT), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    return preds


def wrap_detection(input_image, output_data):
    class_ids = []
    confidences = []
    boxes = []

    rows = output_data.shape[0]

    image_width, image_height, _ = input_image.shape

    x_factor = image_width / INPUT_WIDTH
    y_factor =  image_height / INPUT_HEIGHT

    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.2:

            classes_scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            if (classes_scores[class_id] > .25):

                confidences.append(confidence)

                class_ids.append(class_id)

                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45) 

    result_class_ids = []
    result_confidences = []
    result_boxes = []

    for i in indexes:
        result_confidences.append(confidences[i])
        result_class_ids.append(class_ids[i])
        result_boxes.append(boxes[i])

    return result_class_ids, result_confidences, result_boxes



inputImage = format_yolov5(image)
outs = detect(inputImage, net)

class_ids, confidences, boxes = wrap_detection(inputImage, outs[0])

colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0), (255, 0, 0)]

for (classid, confidence, box) in zip(class_ids, confidences, boxes):
    if confidence >= 0.2:
        color = colors[int(classid) % len(colors)]
        cv2.rectangle(image, box, color, 2)
        cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)
        cv2.putText(image, list_classes[classid], (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0))

from matplotlib import pyplot as plt
import numpy as np
import cv2


img2 = image[:,:,::-1]
plt.imshow(img2)

